In [ ]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys


In [19]:
import matplotlib.pyplot as plt
from matplotlib import colors

import pandas as pd

In [20]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [21]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [22]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [23]:
# Picking table from Crypto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [24]:
def maybe_float(s):
    try:
        return round(float(s.replace(".", "").replace(",", ".")), 2)
    except (ValueError, TypeError):
        return s

In [25]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        data_sort = cell.get_attribute('data-sort')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)

        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:  
                cell_data.append(maybe_float(part_splitted))

        elif data_sort:
            cell_data.append(round(float(data_sort), 2))
            
        else:
            cell_data.append(cell.text)
    return cell_data

In [26]:
def process_rows(rows):
    table_data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [27]:
table_data = process_rows(rows)

In [28]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]

headers[0] = "IMG Coin"
insertions = [(0, "ID"), (3, "$"), (8, "30 d"), (9, "$"), (11, "$"), (13, "FDV")
]
for index, value in insertions:
    headers.insert(index, value)

In [29]:
df = pd.DataFrame(table_data)
df_treated_with_link = df.loc[:, (df != '').any(axis=0)] #test drop_na
df_treated_with_link = df_treated_with_link.drop(df.columns[4], axis=1)
df_treated_with_link = df_treated_with_link.reset_index(drop=True)
df_treated_with_link


,1,2,3,5,6,7,8,9,10,11,12,13,14,15,17
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,US$,66452.32,1.66,2.32,5.21,11.09,US$,3.809498e+10,US$,1.313072e+12,1.395491e+12,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,US$,2694.17,1.90,2.13,5.83,6.70,US$,1.700408e+10,US$,3.244270e+11,3.244185e+11,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,US$,1.00,0.27,0.09,0.33,0.14,US$,5.290223e+10,US$,1.197395e+11,1.196423e+11,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,BNB\nBNB,US$,613.04,1.07,2.65,7.30,13.55,US$,1.432768e+09,US$,8.949522e+10,8.949511e+10,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,Solana\nSOL,US$,160.91,3.16,3.85,8.57,9.71,US$,4.352207e+09,US$,7.547573e+10,9.416222e+10,https://www.coingecko.com/coins/4128/sparkline...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,https://assets.coingecko.com/coins/images/2643...,Starknet\nSTRK,US$,0.48,1.57,0.49,22.32,33.40,US$,1.876564e+08,US$,8.647349e+08,4.854667e+09,https://www.coingecko.com/coins/26433/sparklin...
96,97,https://assets.coingecko.com/coins/images/1775...,Marinade Staked SOL\nMSOL,US$,196.24,2.43,3.98,8.85,9.99,US$,2.571308e+07,US$,8.497417e+08,8.497417e+08,https://www.coingecko.com/coins/17752/sparklin...
97,98,https://assets.coingecko.com/coins/images/3508...,Wormhole\nW,US$,0.32,1.33,15.94,35.85,45.19,US$,1.544668e+08,US$,8.401862e+08,3.256574e+09,https://www.coingecko.com/coins/35087/sparklin...
98,101,https://assets.coingecko.com/coins/images/1307...,Conflux\nCFX,US$,0.19,3.17,12.35,28.79,42.35,US$,1.287116e+08,US$,8.374818e+08,1.046818e+09,https://www.coingecko.com/coins/13079/sparklin...


In [30]:
df_treated_with_link.style\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})\
    .background_gradient(subset=[6])
    #


,1,2,3,5,6,7,8,9,10,11,12,13,14,15,17
0,1,https://assets.coingecko.com/coins/images/1/standard/bitcoin.png?1696501400,Bitcoin BTC,US$,66452.320000,1.660000,2.320000,5.210000,11.090000,US$,38094981973.000000,US$,1313071820824.000000,1395490525948.189941,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/standard/ethereum.png?1696501628,Ethereum ETH,US$,2694.170000,1.900000,2.130000,5.830000,6.700000,US$,17004083344.000000,US$,324427040789.000000,324418480598.640015,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/standard/Tether.png?1696501661,Tether USDT,US$,1.000000,0.270000,0.090000,0.330000,0.140000,US$,52902234201.000000,US$,119739545366.000000,119642277414.839996,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/standard/bnb-icon2_2x.png?1696501970,BNB BNB,US$,613.040000,1.070000,2.650000,7.300000,13.550000,US$,1432768365.000000,US$,89495224767.000000,89495108347.610001,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128/standard/solana.png?1718769756,Solana SOL,US$,160.910000,3.160000,3.850000,8.570000,9.710000,US$,4352206758.000000,US$,75475732673.000000,94162220608.039993,https://www.coingecko.com/coins/4128/sparkline.svg
5,6,https://assets.coingecko.com/coins/images/6319/standard/usdc.png?1696506694,USDC USDC,US$,1.000000,0.270000,0.060000,0.050000,-0.210000,US$,6932284939.000000,US$,36408669834.000000,36386976813.830002,https://www.coingecko.com/coins/6319/sparkline.svg
6,7,https://assets.coingecko.com/coins/images/44/standard/xrp-symbol-white-128.png?1696501442,XRP XRP,US$,0.600000,1.710000,0.830000,2.410000,4.090000,US$,1511499509.000000,US$,33743747007.000000,59661307228.750000,https://www.coingecko.com/coins/44/sparkline.svg
7,8,https://assets.coingecko.com/coins/images/13442/standard/steth_logo.png?1696513206,Lido Staked Ether STETH,US$,2693.590000,1.890000,2.140000,7.060000,8.320000,US$,39417337.000000,US$,26452631896.000000,26431230305.599998,https://www.coingecko.com/coins/13442/sparkline.svg
8,9,https://assets.coingecko.com/coins/images/5/standard/dogecoin.png?1696501409,Dogecoin DOGE,US$,0.130000,3.400000,10.870000,21.610000,26.500000,US$,1776521081.000000,US$,18723591129.000000,18720516641.779999,https://www.coingecko.com/coins/5/sparkline.svg
9,10,https://assets.coingecko.com/coins/images/17980/standard/photo_2024-09-10_17.09.00.jpeg?1725963446,Toncoin TON,US$,6.010000,0.530000,3.600000,6.420000,4.640000,US$,466282955.000000,US$,15285811100.000000,30791429419.860001,https://www.coingecko.com/coins/17980/sparkline.svg


In [31]:
df_treated_with_link.columns = [headers]
#df_treated_with_link

In [32]:
df_treated = df_treated_with_link.drop(columns=['IMG Coin', 'Últimos 7 dias'])

In [33]:
df_treated

,ID,Moeda,$,Preço,1 h,24 h,7 d,30 d,$,Volume em 24 h,$,Capitalização de mercado,FDV
0,1,Bitcoin\nBTC,US$,66452.32,1.66,2.32,5.21,11.09,US$,3.809498e+10,US$,1.313072e+12,1.395491e+12
1,2,Ethereum\nETH,US$,2694.17,1.90,2.13,5.83,6.70,US$,1.700408e+10,US$,3.244270e+11,3.244185e+11
2,3,Tether\nUSDT,US$,1.00,0.27,0.09,0.33,0.14,US$,5.290223e+10,US$,1.197395e+11,1.196423e+11
3,4,BNB\nBNB,US$,613.04,1.07,2.65,7.30,13.55,US$,1.432768e+09,US$,8.949522e+10,8.949511e+10
4,5,Solana\nSOL,US$,160.91,3.16,3.85,8.57,9.71,US$,4.352207e+09,US$,7.547573e+10,9.416222e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Starknet\nSTRK,US$,0.48,1.57,0.49,22.32,33.40,US$,1.876564e+08,US$,8.647349e+08,4.854667e+09
96,97,Marinade Staked SOL\nMSOL,US$,196.24,2.43,3.98,8.85,9.99,US$,2.571308e+07,US$,8.497417e+08,8.497417e+08
97,98,Wormhole\nW,US$,0.32,1.33,15.94,35.85,45.19,US$,1.544668e+08,US$,8.401862e+08,3.256574e+09
98,101,Conflux\nCFX,US$,0.19,3.17,12.35,28.79,42.35,US$,1.287116e+08,US$,8.374818e+08,1.046818e+09


In [34]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx", index=False)